In [1]:
import numpy as np
import tensorflow as tf
import csv

In [2]:
X,Y=[],[]
#reading the file
def getdata(filename):
    with open(filename, 'r') as csvfile:
        csvFileReader = csv.reader(csvfile)
        next(csvFileReader)
        
        for row in csvFileReader:
            X.append([[float(row[0])],[float(row[1])],[float(row[2])],[float(row[3])],[float(row[4])],[float(row[5])],[float(row[6])],[float(row[7])]])
            Y.append(float(row[8]))
    
    return X,Y        

In [3]:
X,Y = getdata('diabetes.csv')
#have the X and Y ready, time to make training and testing set ready 

In [4]:
X=np.reshape(X,(768,8))
Y=np.reshape(Y,(768,1))

#normalizing inputs

X_max = (np.max(X, axis=0))

for i in range(768):
    for j in range(8):
        X[i][j]= X[i][j]/X_max[j]


In [5]:
X_train, X_test, Y_train, Y_test = X[:700],X[700:],Y[:700],Y[700:]
X_train, Y_train = X_train.T,Y_train.T
X_test, Y_test = X_test.T,Y_test.T

In [6]:
print(np.shape(X_train),np.shape(Y_train))
print(np.shape(X_test),np.shape(Y_test))

(8, 700) (1, 700)
(8, 68) (1, 68)


In [23]:

#make all parameters and hyper parameter variables
n_1 = 12
n_2 = 5
n_3 = 5
n_4 = 3
n_out = 1

input_to_nn_X = tf.placeholder(tf.float32, shape=(8,None)) 
input_to_nn_Y = tf.placeholder(tf.float32, shape=(1,None))

In [24]:
#making the neural network
#4-Layer neural net
def neural_network(x):
    tf.set_random_seed(1)
    #initializing weight and bias matrices
    hidden_layer_1 = {'W1': tf.Variable(tf.random_normal([n_1,8]))*0.01,
                      'b1': tf.Variable(tf.zeros([n_1,1]))}
    
    hidden_layer_2 = {'W2': tf.Variable(tf.random_normal([n_2,n_1]))*0.01,
                      'b2': tf.Variable(tf.zeros([n_2,1]))}
    
    #hidden_layer_3 = {'W3': tf.Variable(tf.random_normal([n_3,n_2]))*0.01,
      #                'b3': tf.Variable(tf.zeros([n_3,1]))}
    
    #hidden_layer_4 = {'W4': tf.Variable(tf.random_normal([n_4,n_3]))*0.01,
                      #'b4': tf.Variable(tf.zeros([n_4,1]))}
    
    output_layer = {'W5': tf.Variable(tf.random_normal([n_out,n_2]))*0.01,
                    'b5': tf.Variable(tf.zeros([n_out,1]))}
    
    #computing neurons' matrix multiplications
    #passing through activation functions relu-->relu-->relu-->sigmoid
    Z1 = tf.add(tf.matmul(hidden_layer_1['W1'],input_to_nn_X),hidden_layer_1['b1'])
    A1 = tf.nn.relu(Z1)
   
    Z2 = tf.matmul(hidden_layer_2['W2'],A1)+ hidden_layer_2['b2']
    A2 = tf.nn.relu(Z2)
    
    #Z3 = tf.matmul(hidden_layer_3['W3'],A2)+ hidden_layer_3['b3']
    #A3 = tf.nn.relu(Z3)
    
    #Z4 = tf.matmul(hidden_layer_4['W4'],A3)+ hidden_layer_4['b4']
    #A4 = tf.nn.relu(Z4)
    
    Z5 = tf.matmul(output_layer['W5'],A2) + output_layer['b5']
    A5 = tf.nn.sigmoid(Z5)
    
    return A5
    

In [27]:
def train_neural_network(x):
    prediction = neural_network(x)
    cost = tf.losses.mean_squared_error(input_to_nn_Y,prediction)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.012).minimize(cost)
    
    epochs = 500000
    hm_correct=0
    hm_training_correct=0
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(epochs):
            epoch_loss = 0
            _,c = sess.run([optimizer,cost], feed_dict={input_to_nn_X:X_train,
                                                  input_to_nn_Y:Y_train})
            epoch_loss+=c
            if epoch%10000==0:
                print('Epoch', epoch+10000,'completed out of ',epochs,' loss:', epoch_loss)
            
        correct = sess.run(prediction, feed_dict={input_to_nn_X:X_test,
                                                  input_to_nn_Y:Y_test})
        
        #calculating the accuracy of our model for training set and testing set
        for i in range(68):
            if correct[0][i]<0.5:
                correct[0][i]=0
            else:
                correct[0][i]=1
                
            if correct[0][i]==Y_test[0][i]:
                hm_correct+=1
        
        training_correct = sess.run(prediction,feed_dict={input_to_nn_X:X_train,
                                                  input_to_nn_Y:Y_train})
        for i in range(700):
            if training_correct[0][i]<0.5:
                training_correct[0][i]=0
            else:
                training_correct[0][i]=1
                
            if training_correct[0][i]==Y_train[0][i]:
                hm_training_correct+=1
        print('Training accuracy: ' + str((hm_training_correct/700)*100)+ '%')        
        print('Testing accuracy : '+str((hm_correct/68)*100) + '%')

In [28]:
train_neural_network(input_to_nn_X)

Epoch 10000 completed out of  500000  loss: 0.25
Epoch 20000 completed out of  500000  loss: 0.15156321227550507
Epoch 30000 completed out of  500000  loss: 0.15152208507061005
Epoch 40000 completed out of  500000  loss: 0.1515152007341385
Epoch 50000 completed out of  500000  loss: 0.1514354944229126
Epoch 60000 completed out of  500000  loss: 0.14930619299411774
Epoch 70000 completed out of  500000  loss: 0.14739125967025757
Epoch 80000 completed out of  500000  loss: 0.14701074361801147
Epoch 90000 completed out of  500000  loss: 0.14659591019153595
Epoch 100000 completed out of  500000  loss: 0.14613506197929382
Epoch 110000 completed out of  500000  loss: 0.1460821032524109
Epoch 120000 completed out of  500000  loss: 0.14607302844524384
Epoch 130000 completed out of  500000  loss: 0.14607658982276917
Epoch 140000 completed out of  500000  loss: 0.14607365429401398
Epoch 150000 completed out of  500000  loss: 0.14607253670692444
Epoch 160000 completed out of  500000  loss: 0.14607